In [151]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [152]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

In [153]:
path='imdb.csv'

In [154]:
try:
    load_df = pd.read_csv(path,sep=';')
    where=load_df['Id'].iloc[-1]+1
except:
  where=1


In [155]:
def keywords(id):
    word=''
    key_url='https://www.imdb.com/title/tt'+id+'/keywords/'
    response_keyword=requests.get(key_url,headers=headers)
    content_keyword=response_keyword.content
    soup_keyword=BeautifulSoup(content_keyword,'html.parser')
    keyword_elements=soup_keyword.find_all('a',class_='ipc-metadata-list-summary-item__t')
    for element in keyword_elements:
     word += ', ' + element.text
    #print('keyword')
    return word[2:len(word)]

In [267]:
def release_info(soup):
   date='NaN'
   x= soup.find_all('li',class_='ipc-metadata-list__item ipc-metadata-list-item--link')
   for r in x:
      if 'Release date' in r.text:
       date=r.text.replace('Release date','')
       break
   return date

In [158]:
df={'Id':[],'Title':[],'Storyline':[],'Genres':[],'Director':[],'Date':[],'Runtime':[],'Rating':[],'Color':[],'Country':[],'keywords':[]}

In [159]:
def country(soup):
    r_items = soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    inde=0
    for r in r_items:
  
      if r.text == 'None':
        country_name= r_items[inde-1].text
        break
      else:
        country_name='NaN'
      inde=inde+1
    return country_name

In [160]:
movie_genres = ["Action","Adventure","Comedy","Drama","Horror","Science Fiction (Sci-Fi)","Fantasy","Romance","Mystery","Crime","Thriller","Animation","Musical","War","Western","Biography","Historical",
    "Documentary","Family","Sports","Superhero","Musical Comedy","Film Noir","Romantic Comedy (Rom-Com)","Fantasy Adventure","Action Comedy","Psychological Thriller","Science Fantasy",
    "Epic","Post-Apocalyptic","History"]

In [161]:
def genere_fun(soup):
 gen_list=' '
 try:
  genres = soup.find_all('span', class_='ipc-chip__text')
  for g in  genres:
    if g.text in movie_genres:
        gen_list=g.text+' '+gen_list
  return gen_list 
 except:
  return 'NaN'
    

In [233]:
def runtime_(soup):
 runtime_test=soup.find_all('li',class_='ipc-metadata-list__item')
 color='NaN'
 runtime="NaN"
 for i in runtime_test:
    if 'Runtime' in i.text:
     runtime=i.text.replace('Runtime','')   
    if 'Color' in i.text:
      color=i.text #.replace('Color',' ')
 return runtime,color

In [296]:
# function to test all other function

where =1517268
for num in range(where,where-10,-1):
  len_num=7-len(str(num))
  id_='0'*len_num+str(num)
  w_url='https://www.imdb.com/title/tt'+id_+'/'
  response=requests.get(w_url,headers=headers)
  content=response.content
  soup=BeautifulSoup(content,'html.parser')
  #print(w_url,keywords_(soup))

  #break

Adventure
Comedy
Fantasy
Back to top
https://www.imdb.com/title/tt1517268/ None
Documentary
Short
Back to top
https://www.imdb.com/title/tt1517267/ None
Action
Sci-Fi
Thriller
Back to top
https://www.imdb.com/title/tt1517266/ None
Documentary
Back to top
https://www.imdb.com/title/tt1517265/ None
Back to top
https://www.imdb.com/title/tt1517264/ None
Music
Back to top
https://www.imdb.com/title/tt1517263/ None
Comedy
Drama
Back to top
https://www.imdb.com/title/tt1517262/ None
Documentary
Back to top
https://www.imdb.com/title/tt1517261/ None
Action
Adventure
Drama
Back to top
https://www.imdb.com/title/tt1517260/ None
Documentary
Back to top
https://www.imdb.com/title/tt1517259/ None


In [ ]:
%%time
i=1
start_time=time.time()
for num in range(where,100000000):
                            len_num=7-len(str(num))
                            id_='0'*len_num+str(num)
                            w_url='https://www.imdb.com/title/tt'+id_+'/'
                            try:
                              response=requests.get(w_url,headers=headers)
                              content=response.content
                              soup=BeautifulSoup(content,'html.parser')
                              Title=soup.find('span',class_='sc-afe43def-1 fDTGTb').text
                            except:
                              print(i,w_url,'>> Error')
                              continue
                            print(i,w_url,'<< processing')
                            Title=soup.find('span',class_='sc-afe43def-1 fDTGTb').text
                            Director=soup.find_all('a',class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')[0].text
                            Date=release_info(soup)
                            
                            Runtime_,colr=runtime(id_)
                            try:
                             Rating=(soup.find_all('span',class_='sc-bde20123-1 iZlgcd')[0]).text
                            except:
                              Rating="NaN"
                            country_=country(soup)
                            df['Country'].append(country_)
                            key_words=keywords(str(id_))
                            i=i+1
                            df['Id'].append(id_)
                            df['Link'].append(w_url)
                            df['Genres'].append(genere_fun(soup))
                            df['Title'].append(Title)
                            df['Director'].append(Director)
                            df['Date'].append(Date)
                            df['Runtime'].append(Runtime_)
                            df['Color'].append(colr)
                            df['Rating'].append(Rating)
                            df['keywords'].append(key_words)
                            if i==100:
                              try:
                               main_df=pd.DataFrame(df)
                               save_df=pd.concat([load_df, main_df], ignore_index=True)
                               save_df.to_csv(path,index=False,sep=';',header=True)
                               load_df = pd.read_csv(path,sep=';')
                               print('>>',load_df.shape)
                               df={'Id':[],'Link':[],'Title':[],'Storyline':[],'Director':[],'Date':[],'Runtime':[],'Rating':[],'Color':[],'Country':[],'keywords':[]}
                               end_time=time.time()
                               print('>>Time', (end_time-start_time)/100)
                               start_time=time.time()
                              except:
                                main_df=pd.DataFrame(df)
                                save_df=main_df
                                save_df.to_csv(path,index=False,sep=';',header=True)
                                load_df = pd.read_csv(path,sep=';')
                                print('>>',load_df.shape)
                                df={'Id':[],'Link':[],'Title':[],'Storyline':[],'Director':[],'Date':[],'Runtime':[],'Rating':[],'Color':[],'Country':[],'keywords':[]}
                                print('File not found Solved')

                              i=1
                          